# 向量化字符串操作(Vectorized String Operations)
使用 Python 的一个优势就是字符串处理起来比较容易。在此基础上创建的 Pandas 同样提 供了一系列向量化字符串操作(vectorized string operation),它们都是在处理(清洗)现实 工作中的数据时不可或缺的功能。在这一节中,我们将介绍 Pandas 的字符串操作,学习如 何用它们对一个从网络采集来的杂乱无章的数据集进行局部清理。

## Pandas 字符串操作简介(Introducing Pandas String Operations)
前面的章节已经介绍过如何用 NumPy 和 Pandas 进行一般的运算操作,因此我们也能简便快速地对多个数组元素执行同样的操作,例如:

In [1]:
import numpy as np
import pandas as pd

x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

向量化操作简化了纯数值的数组操作语法——我们不需要再担心数组的长度或维度,只需要关心需要的操作。然而,由于 NumPy 并没有为字符串数组提供简单的接口,因此需要通过繁琐的 for 循环来解决问题:

In [2]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

虽然这么做对于某些数据可能是有效的,但是假如数据中出现了缺失值,那么这样做就会引起异常,例如:

In [3]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
try:
    [s.capitalize() for s in data]
except Exception as e:
    print(e)

'NoneType' object has no attribute 'capitalize'


Pandas 为包含字符串的 Series 和 Index 对象提供的 str 属性堪称两全其美的方法,它既可以满足向量化字符串操作的需求,又可以正确地处理缺失值。例如,我们用前面的数据data 创建了一个 Pandas 的 Series :

In [4]:
names = pd.Series(data)
names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

现在就可以直接调用转换大写方法 capitalize() 将所有的字符串变成大写形式,缺失值会被跳过:

In [5]:
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

在 str 属性后面用 Tab 键,可以看到 Pandas 支持的所有向量化字符串方法。

## Pandas 字符串方法列表(Tables of Pandas String Methods)
如果你熟悉 Python 的字符串方法的话,就会发现 Pandas 绝大多数的字符串语法都很直观,甚至可以列成一个表格。在深入论述后面的内容之前,让我们先从这一步开始。这一节的示例将采用一些人名来演示:

In [6]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

#### 1. 与Python字符串方法相似的方法
几乎所有 Python 内置的字符串方法都被复制到 Pandas 的向量化字符串方法中。下面的表格列举了 Pandas 的 str 方法借鉴 Python 字符串方法的内容:

| fun      | fun          | fun          | fun          |
| -------- | ------------ | ------------ | ------------ |
| len()    | lower()      | translate()  | islower()    |
| ljust()  | upper()      | startswith() | isupper()    |
| rjust()  | find()       | endswith()   | isnumeric()  |
| center() | rfind()      | isalnum()    | isdecimal()  |
| zfill()  | index()      | isalpha()    | split()      |
| strip()  | rindex()     | isdigit()    | rsplit()     |
| rstrip() | capitalize() | isspace()    | partition()  |
| lstrip() | swapcase()   | istitle()    | rpartition() |

需要注意的是,这些方法的返回值不同,例如 lower() 方法返回一个字符串 Series :

In [7]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

但是有些方法返回数值:

In [8]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

有些方法返回布尔值:

In [9]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

还有些方法返回列表或其他复合值:

In [10]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

在接下来的内容中,我们将进一步学习这类由列表元素构成的 Series (series-of-lists)对象。

#### 2. 使用正则表达式的方法
还有一些支持正则表达式的方法可以用来处理每个字符串元素。表中的内容是 Pandas向量化字符串方法根据 Python 标准库的 re 模块函数实现的 API。

Pandas向量化字符串方法与Python标准库的 re 模块函数的对应关系

| Method     | Description                                                         |
| ---------- | ------------------------------------------------------------------- |
| match()    | Callre.match()on each element, returning a boolean.                 |
| extract()  | Callre.match()on each element, returning matched groups as strings. |
| findall()  | Callre.findall()on each element                                     |
| replace()  | Replace occurrences of pattern with some other string               |
| contains() | Callre.search()on each element, returning a boolean                 |
| count()    | Count occurrences of pattern                                        |
| split()    | Equivalent tostr.split(), but accepts regexps                       |
| rsplit()   | Equivalent tostr.rsplit(), but accepts regexps                      |


| 方法       | 描述                                                    |
| ---------- | ------------------------------------------------------- |
| match()    | 对每个元素调用 re.match() ,返回布尔类型值               |
| extract()  | 对每个元素调用 re.match() ,返回匹配的字符串组( groups ) |
| findall()  | 对每个元素调用 re.findall()                             |
| replace()  | 用正则模式替换字符串                                    |
| contains() | 对每个元素调用 re.search() ,返回布尔类型值              |
| count()    | 计算符合正则模式的字符串的数量                          |
| split()    | 等价于 str.split() ,支持正则表达式                      |
| rsplit()   | 等价于 str.rsplit() ,支持正则表达式                     |

通过这些方法,你就可以实现各种有趣的操作了。例如,可以提取元素前面的连续字母作为每个人的名字(first name):

In [11]:
monte.str.extract('([A-Za-z]+)')

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


In [12]:
#A pattern with two groups will return a DataFrame with two columns.Non-matches will be NaN.
s = pd.Series(['a1', 'b2', 'c3'])
s.str.extract(r'([ab])(\d)')

,0,1
0,a,1
1,b,2
2,NaN,NaN


In [13]:
#Named groups will become column names in the result.
s.str.extract(r'(?P<letter>[ab])(?P<digit>\d)')

,letter,digit
0,a,1
1,b,2
2,NaN,NaN


我们还能实现更复杂的操作,例如找出所有开头和结尾都是辅音字母的名字——这可以用正则表达式中的开始符号( ^ )与结尾符号( $ )来实现:

In [14]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

In [15]:
monte.str.extract(r'^([^AEIOU].*[^aeiou])$') #extract方法必须包含分组

,0
0,Graham Chapman
1,NaN
2,Terry Gilliam
3,NaN
4,Terry Jones
5,Michael Palin


In [16]:
monte.str.match(r'^[^AEIOU].*[^aeiou]$')

0     True
1    False
2     True
3    False
4     True
5     True
dtype: bool

#### 3. 其他字符串方法
还有其他一些方法也可以实现方便的操作(如表所示)。

其他Pandas字符串方法

| Method          | Description                                                       |
| --------------- | ----------------------------------------------------------------- |
| get()           | Index each element                                                |
| slice()         | Slice each element                                                |
| slice_replace() | Replace slice in each element with passed value                   |
| cat()           | Concatenate strings                                               |
| repeat()        | Repeat values                                                     |
| normalize()     | Return Unicode form of string                                     |
| pad()           | Add whitespace to left, right, or both sides of strings           |
| wrap()          | Split long strings into lines with length less than a given width |
| join()          | Join strings in each element of the Series with passed separator  |
| get_dummies()   | extract dummy variables as a dataframe |

| 方法            | 描述                                                                    |
| --------------- | ----------------------------------------------------------------------- |
| get()           | 获取元素索引位置上的值,索引从 0 开始                                    |
| slice()         | 对元素进行切片取值                                                      |
| slice_replace() | 对元素进行切片替换                                                      |
| cat()           | 连接字符串(此功能比较复杂,建议阅读文档)                                 |
| repeat()        | 重复元素                                                                |
| normalize()     | 将字符串转换为 Unicode 规范形式                                         |
| pad()           | 在字符串的左边、右边或两边增加空格                                      |
| wrap()          | 将字符串按照指定的宽度换行                                              |
| join()          | 用分隔符连接 Series 的每个元素                                          |
| get_dummies()   | 按照分隔符提取每个元素的 dummy 变量,转换为独热(one-hot)编码的 DataFrame |

##### (1) 向量化字符串的取值与切片操作。
这里需要特别指出的是, get() 与 slice() 操作可以从每个字符串数组中获取向量化元素。例如,我们可以通过 str.slice(0, 3) 获取每个字符串数组的前三个字符。通过 Python 的标准取值方法也可以取得同样的效果,例如df.str.slice(0, 3) 等价于 df.str[0:3] :

In [17]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

In [18]:
monte.str.slice(0, 3)

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

df.str.get(i) 与 df.str[i] 的按索引取值效果类似。

In [19]:
monte.str[0]

0    G
1    J
2    T
3    E
4    T
5    M
dtype: object

get() 与 slice() 操作还可以在 split() 操作之后使用。例如,要获取每个姓名的姓(last name),可以结合使用 split() 与 get() :

In [20]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

In [21]:
monte.str.split().str[-1]

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

##### (2) 指标变量
另一个需要多花点儿时间解释的是 get_dummies() 方法。当你的数据有一列 包含了若干已被编码的指标(coded indicator)时,这个方法就能派上用场了。例如, 假设有一个包含了某种编码信息的数据集,如 A= 出生在美国、B= 出生在英国、C= 喜 欢奶酪、D= 喜欢午餐肉:

In [22]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


get_dummies() 方法可以让你快速将这些指标变量分割成一个独热编码的 DataFrame (每个元素都是 0 或 1):

In [23]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


通过 Pandas 自带的这些字符串操作方法,你就可以建立一个功能无比强大的字符串处 理程序来清洗自己的数据了。

虽然本书将不再继续介绍这些方法,但是希望你仔细阅读 Pandas 在线文档中“Working with Text Data”(http://pandas.pydata.org/pandas-docs/stable/text.html)节的相关资源。

## 案例 : 食谱数据库(Example: Recipe Database)
前面介绍的这些向量化字符串操作方法非常适合用来处理现实中那些凌乱的数据。下面将 通过一个从不同网站获取的公开食谱数据库的案例来进行演示。我们的目标是将这些食谱 数据解析为食材列表,这样就可以根据现有的食材快速找到食谱。

获取数据的脚本可以在 https://github.com/fictivekin/openrecipes 上找到,那里还有最新版的 数据库链接。(已经连接不到了，用这个：https://github.com/sameergarg/scala-elasticsearch/blob/master/conf/recipeitems-latest.json.gz)

截至 2016 年春,这个数据集已经有 30MB 了。可以通过下面的命令下载并解压数据:

In [24]:
# !curl -O http://openrecipes.s3.amazonaws.com/recipeitems-latest.json.gz
# !gunzip recipeitems-latest.json.gz

这个数据库是 JSON 格式的,来试试通过 pd.read_json 读取数据:

In [25]:

try:
    recipes = pd.read_json('recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


糟糕!我们得到的竟然是提示数据里有“trailing data”(数据断行)的 ValueError 错误。从网上搜索这个错误,得知原因好像是虽然文件中的每一行都是一个有效的 JSON 对象,但是全文却不是这样。来看看文件是不是这样:

In [26]:
with open('recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(line).shape

(2, 12)

显然每一行都是一个有效的 JSON 对象,因此需要将这些字符串连接在一起。解决这个问题的一种方法就是新建一个字符串,将所有行 JSON 对象连接起来,然后再通过 pd.read_json 来读取所有数据:

In [27]:
# read the entire file into a Python array# 将文件内容读取成Python数组
with open('recipeitems-latest.json', 'r') as f:
    # Extract each line# 提取每一行内容
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list# 将所有内容合并成一个列表
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON# 用JSON形式读取数据
recipes = pd.read_json(data_json)

In [28]:
recipes.shape

(173278, 17)

In [29]:
recipes.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173278 entries, 0 to 173277
Data columns (total 17 columns):
_id                   173278 non-null object
cookTime              117936 non-null object
creator               395 non-null object
dateModified          161 non-null object
datePublished         78110 non-null object
description           158068 non-null object
image                 158278 non-null object
ingredients           173278 non-null object
name                  173278 non-null object
prepTime              130186 non-null object
recipeCategory        388 non-null object
recipeInstructions    4 non-null object
recipeYield           165628 non-null object
source                173278 non-null object
totalTime             1570 non-null object
ts                    173278 non-null object
url                   173278 non-null object
dtypes: object(17)
memory usage: 320.1 MB


这样就会看到将近 20 万份食谱,共 17 列。抽一行看看具体内容:

In [30]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

这里有一堆信息,而且其中有不少都和从网站上抓取的数据一样,字段形式混乱。值得关注的是,食材列表是字符串形式,我们需要从中抽取感兴趣的信息。下面来仔细看看这个字段:

In [31]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

食材列表平均 250 个字符,最短的字符串是 0,最长的竟然接近 1 万字符!
出于好奇心,来看看这个拥有最长食材列表的究竟是哪道菜:

In [32]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

/home/lifengchao/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [33]:
ingredients_max_len = recipes.ingredients.str.len().max()

In [34]:
recipes.query("ingredients.str.len()==@ingredients_max_len").iloc[0]["name"]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

从名字就可以看出,这绝对是个复杂的食谱。我们还可以再做一些累计探索,例如看看哪些食谱是早餐:

In [35]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

或者看看有多少食谱用肉桂(cinnamon)作为食材:

In [36]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

还可以看看究竟是哪些食谱里把肉桂错写成了“cinamon”:

In [37]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

这些基本的数据探索都可以用 Pandas 的字符串工具来处理,Python 非常适合进行类似的数据清理工作。

#### 1. 制作简易的美食推荐系统
现在让我们更进一步,来制作一个简易的美食推荐系统:如果用户提供一些食材,系统 就会推荐使用了所有食材的食谱。这说起来是容易,但是由于大量不规则(heterogeneity) 数据的存在,这个任务变得十分复杂,例如并没有一个简单直接的办法可以从每一行数据 中清理出一份干净的食材列表。因此,我们在这里简化处理:首先提供一些常见食材列 表,然后通过简单搜索判断这些食材是否在食谱中。为了简化任务,这里只列举常用的香 料和调味料:

In [38]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

现在就可以通过一个由 True 与 False 构成的布尔类型的 DataFrame 来判断食材是否出现在某个食谱中:

In [39]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


现在,来找一份使用了欧芹(parsley)、辣椒粉(paprika)和龙蒿叶(tarragon)这三种食材的食谱。我们可以通过DataFrame 的 query() 方法来快速完成计算:

In [40]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

最后只找到了十份同时包含这三种食材的食谱,让我们用索引看看究竟是哪些食谱:

In [41]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

现在已经将搜索范围缩小到了原来近两万份食谱的两千分之一了,这样就可以从这个小集合中精挑细选出中意的食谱。

#### 2. 继续完善美食推荐系统
希望这个示例能让你对 Pandas 字符串方法可以高效解决哪些数据清理问题有个初步概念。 当然,如果要建立一个稳定的美食推荐系统,还需要做大量的工作!从每个食谱中提取完 整的食材列表是这个任务的重中之重。不过,由于食材的书写格式千奇百怪,解析它们需 要耗费大量时间。这其实也揭示了数据科学的真相——真实数据的清洗与整理工作往往会 占据的大部分时间,而使用 Pandas 提供的工具可以提高你的工作效率。

# String Manipulation（字符串处理）
python很多内建方法很适合处理string。而且对于更复杂的模式，可以配合使用正则表达式。而pandas则混合了两种方式。
## String Object Methods（字符串对象方法）
大部分string处理，使用内建的一些方法就足够了。比如，可以用split来分割用逗号区分的字符串：

In [42]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

split经常和strip一起搭配使用来去除空格（包括换行符）:

In [43]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

join方法连接字符串列表

In [44]:
'::'.join(pieces)

'a::b::guido'

其他一些方法适合锁定子字符串位置相关的。用in关键字是检测substring最好的方法，当然，index和find也能完成任务：

In [45]:
'guido' in val

True

In [46]:
val.index(',')

1

注意index和find的区别。如果要找的string不存在的话，index会报错。而find会返回-1:

In [47]:
val.find(':')

-1

count会返回一个substring出现的次数：

In [48]:
val.count(',')

2

replace会取代一种出现方式（pattern）。也通常用于删除pattern，传入一个空字符串即可：

In [49]:
val.replace(',', '::')

'a::b:: guido'

In [50]:
val.replace(',', '')

'ab guido'

这里一些内建的string方法：(暂时没有)

## Regular Expressions（正则表达式）
正则表达式能让我们寻找更复杂的pattern。通常称一个表达式为regex，由正则表达语言来代表一个字符串模式。可以使用python内建的re模块来使用。

关于正则表达式，有很多教学资源，可以自己找几篇来学一些，这里不会介绍太多。

re模块有以下三个类别：patther matching（模式匹配）, substitution（替换）, splitting（分割）。通常这三种都是相关的，一个regex用来描述一种pattern，这样会有很多种用法。这里举个例子，假设我们想要根据空格（tabs，spaces，newlines）来分割一个字符串。用于描述一个或多个空格的regex是\s+:

In [51]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

当调用re.split('\s+', text)的时候，正则表达式第一次被compile编译，并且split方法会被调用搜索text。我们可以自己编译regex，用re.compile，可以生成一个可以多次使用的regex object：

In [52]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

如果想要得到符合regex的所有结果，以一个list结果返回，可以使用findall方法：

In [53]:
regex.findall(text)

['    ', '\t ', '  \t']

为了防止\在正则表达式中的逃逸，推荐使用raw string literal，比如r'C:\x'，而不是使用'C:\\x

使用re.compile创建一个regex object是被强烈推荐的，如果你打算把一个表达式用于很多string上的话，这样可以节省CPU的资源。

match和search，与findall关系紧密。不过findall会返回所有匹配的结果，而search只会返回第一次匹配的结果。更严格地说，match只匹配string开始的部分。这里举个例子说明，我们想要找到所有的邮件地址：

In [54]:
text = """Dave dave@google.com 
          Steve steve@gmail.com 
          Rob rob@gmail.com 
          Ryan ryan@yahoo.com """

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE makes the regex case-insensitive 
regex = re.compile(pattern, flags=re.IGNORECASE)

使用findall找到一组邮件地址：

In [55]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search返回text中的第一个匹配结果。match object能告诉我们找到的结果在text中开始和结束的位置：

In [56]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [57]:
text[m.start():m.end()]

'dave@google.com'

In [58]:
m.group()

'dave@google.com'

regex.match返回None，因为它只会在pattern存在于stirng开头的情况下才会返回匹配结果：

In [59]:
print(regex.match(text))

None


而sub返回一个新的string，把pattern出现的地方替换为我们指定的string：

In [60]:
print(regex.sub('REDACTED', text))

Dave REDACTED 
          Steve REDACTED 
          Rob REDACTED 
          Ryan REDACTED 


假设你想要找到邮件地址，同时，想要把邮件地址分为三个部分，username, domain name, and domain suffix.（用户名，域名，域名后缀）。需要给每一个pattern加一个括号：

In [61]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

match object会返回一个tuple，包含多个pattern组份，通过groups方法：

In [62]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

findall会返回a list of tuples:

In [63]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub也能访问groups的结果，不过要使用特殊符号 \1, \2。\1表示第一个匹配的group，\2表示第二个匹配的group，以此类推：

In [64]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com 
          Steve Username: steve, Domain: gmail, Suffix: com 
          Rob Username: rob, Domain: gmail, Suffix: com 
          Ryan Username: ryan, Domain: yahoo, Suffix: com 


这里给一些正则表达式的方法：(暂时没有)

## Vectorized String Functions in pandas（pandas中的字符串向量化函数）
一些复杂的数据清理中，string会有缺失值：

In [65]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [66]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

可以把一些字符串方法和正则表达式（用lambda或其他函数）用于每一个value上，通过data.map，但是这样会得到NA(null)值。为了解决这个问题，series有一些数组导向的方法可以用于字符串操作，来跳过NA值。这些方法可以通过series的str属性；比如，我们想检查每个电子邮箱地址是否有'gmail' with str.contains:

In [67]:
data.str

In [68]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

正则表达式也可以用，配合任意的re选项，比如IGNORECASE：

In [69]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [70]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

有很多方法用于向量化。比如str.get或index索引到str属性：

In [71]:
matche = data.str.match(pattern, flags=re.IGNORECASE)
matche

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [72]:
matches = data.str.findall(pattern, flags=re.IGNORECASE)
matches

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

为了访问嵌套list里的元素，我们可以传入一个index给函数：

In [73]:
matches.str.get(0)

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [74]:
type(matches.iloc[0])

list

In [75]:
matches.str.get(0).str.get(0)

Dave      dave
Steve    steve
Rob        rob
Wes        NaN
dtype: object

也可以使用这个语法进行切片：

In [76]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

这里有一些字符串向量化的方法：(暂时没有)